<a href="https://colab.research.google.com/github/maroof13/SST-processing/blob/main/maroof_bhai_nc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Necessary Imports

import netCDF4 as nc
import numpy as np
import pandas as pd
import xarray as xr
import os

In [ ]:
# ------------- IGNORE THESE BUT DONT REMOVE (MIGHT HELP LATER) --------------

# path_to_tws = "/content/drive/MyDrive/Data_Rushrukh/GRACE-Data/GRCTellus.JPL.200204_202207.GLO.RL06M.MSCNv02CRI.nc"
# path_to_sst = "/content/ersst.v5.201701.nc"

# # data_tws = nc.Dataset(path_to_tws)
# data_sst = nc.Dataset(path_to_sst)

# print(data_sst['sst'].shape)
# for i in range(data_tws['lwe_thickness'].shape[0]):
#   x = data_tws['lwe_thickness'][i][:][:]
#   plt.imshow(x, interpolation='nearest')
#   plt.savefig('test_image_' + str(i))
#   plt.show()

#   if(i >= 20):
#     break

# print(data_sst)
# print(data_sst['sst'].shape)

# for i in range(data_sst['sst'].shape[0]):
#   x = data_sst['sst'][i][:][:]
#   plt.imshow(x, interpolation='nearest')
#   plt.savefig('test_image_' + str(i))
#   plt.show()

#   if(i >= 20):
#     break

# !wget -r -p -e robots=off -nH --cut-dirs=5 "*.nc" https://www1.ncdc.noaa.gov/pub/data/cmb/ersst/v5/netcdf/

# sample_nc_path = "/content/netcdf/ersst.v5.201701.nc"
# sample_nc_file = nc.Dataset(sample_nc_path)
# print(sample_nc_file['sst'].shape)
# print(type(sample_nc_file['sst']))

# using xarray to read netcdf

# df.iloc[2300]

(1, 1, 89, 180)


In [ ]:
# ----------- starting here --------------

def clean_matrix(path): 
  """
  Parameter: path to one particular NetCDF File
  Reads the NetCDF file, converts to csv, read back into dataframe, drops lev, time, ssta.
  Returns the clean DataFrame
  """
  ds = xr.open_dataset(path)
  df = ds.to_dataframe()
  file_name = 'sample.csv'
  df.to_csv(file_name)
  df = pd.read_csv('/content/sample.csv')
  df = df.drop(["lev", "time", "ssta"], axis=1)

  return df

def extract_year_month(input):
  """
  Input: NetCDF File Name
  Extracts the Year and Month from Filename
  Returns Year and Month
  """

  date = input.split(".")[-2]
  year = date[:4]
  month = date[4:]
  month = str(int(month))

  return year, month

def calculate_mean(month_data):
  """
  Input: month_data - contains all dataframes for all year for one specific month
  Picks the "sst" column from all dataframes
  Calculates the "row-wise" mean for all columns
  Returns the Mean
  """

  sst_mean = pd.DataFrame()
  sst_mean = pd.concat(month_data, axis=1, join='inner')
  sst_mean = sst_mean['sst']
  sst_mean = sst_mean.mean(axis=1)

  return sst_mean

def prepare_data(path):

  """
  Input: path - location of the folder that contains all NetCDF Files
  Reads file one by one
  Extracts Year and Month
  Cleans the respective NetCDF files
  Creates a dictionary for All Years, stores in master_storage

  Returns master_storage
  """
  directory = os.fsencode(path)

  master_storage = {}
  for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".nc"):
      file_path = os.path.join(directory.decode(), filename)
      year, month = extract_year_month(filename)
      year_key = year
      month_key = month

      if(year_key in master_storage.keys()):
        master_storage[year_key].update({month_key: clean_matrix(file_path)})
      else:
        master_storage[year_key] = {
          month_key: clean_matrix(file_path)
        }
  
  return master_storage

def remove_seasonality(master_storage):
  for i in range(12):
    current_month = i + 1

    month_data = []
    for key in master_storage.keys():
      month_data.append(master_storage[key][str(current_month)])
    
    sst_mean = calculate_mean(month_data)
    
    for key in master_storage.keys():
      # print(f"unique before: {master_storage[key][str(current_month)]['sst'].unique()}")
      master_storage[key][str(current_month)]['sst'] = master_storage[key][str(current_month)]['sst'] - sst_mean #switch subtraction order here if needed
      # print(f"unique after: {master_storage[key][str(current_month)]['sst'].unique()}")

  # print(master_storage["2017"]["1"])
  return master_storage

In [ ]:
def prepare_data_and_remove_seasonality(path):
  master_storage = {}
  master_storage = prepare_data(path)
  master_storage = remove_seasonality(master_storage)

  return master_storage

In [ ]:
master_storage = prepare_data_and_remove_seasonality('/content')

dict_keys(['4', '11', '10', '9', '5', '1', '8', '12', '3', '6', '7', '2'])


In [ ]:
print(master_storage)

{'2017': {'4':         lat    lon  sst
0     -88.0    0.0  NaN
1     -88.0    2.0  NaN
2     -88.0    4.0  NaN
3     -88.0    6.0  NaN
4     -88.0    8.0  NaN
...     ...    ...  ...
16015  88.0  350.0  0.0
16016  88.0  352.0  0.0
16017  88.0  354.0  0.0
16018  88.0  356.0  0.0
16019  88.0  358.0  0.0

[16020 rows x 3 columns], '11':         lat    lon  sst
0     -88.0    0.0  NaN
1     -88.0    2.0  NaN
2     -88.0    4.0  NaN
3     -88.0    6.0  NaN
4     -88.0    8.0  NaN
...     ...    ...  ...
16015  88.0  350.0  0.0
16016  88.0  352.0  0.0
16017  88.0  354.0  0.0
16018  88.0  356.0  0.0
16019  88.0  358.0  0.0

[16020 rows x 3 columns], '10':         lat    lon       sst
0     -88.0    0.0       NaN
1     -88.0    2.0       NaN
2     -88.0    4.0       NaN
3     -88.0    6.0       NaN
4     -88.0    8.0       NaN
...     ...    ...       ...
16015  88.0  350.0  0.044404
16016  88.0  352.0  0.042250
16017  88.0  354.0  0.039525
16018  88.0  356.0  0.038915
16019  88.0  358.0  0.03